# <font color='blue'>Banco de Dados Não Relacional</font>
## <font color='blue'>Projeto Final - Script Python para Ingestão de Dados </font>
### <font color='blue'>Rafael Guiselli Felippe</font>

## Estratégia da Solução

- **1**. Criação das tabelas de dimensão no SQL Server
- **2**. Carregar o arquivo JSON para o MongoDB
- **3**. Realizar as transformações necessárias
- **4**. Disponibilizar o arquivo estruturado em uma tabela no SQL Server

In [1]:
# Bibliotecas necessárias
import json
import pandas as pd
import pymongo
import pyodbc

## 1. Criação das tabelas de dimensão no SQL Server

Por se tratar de tabelas com pouca movimentação de dados, elas serão criadas de maneira manual diretamente no SQL Server.

## 2. Carregar o arquivo JSON para o MongoDB

In [2]:
# Conexao ao MongoDB
conexao = pymongo.MongoClient("mongodb://localhost:27017/")

# Definindo o banco de dados
database = "FabricaDB"

if database in conexao.list_database_names():
    db = conexao[database]
    print(f"Conectado ao banco de dados '{database}'.")
else:
    db = conexao[database]
    print(f"Banco de dados '{database}' criado com sucesso.")

# Definindo a coleção      
colecao = "dados_sensores"

if colecao in db.list_collection_names():
    db[colecao].delete_many({})
    print(f"Documentos existentes removidos da coleção '{colecao}'.")
else:
    db.create_collection(colecao)
    print(f"Coleção '{colecao}' criada com sucesso.")

# Carregando o arquivo JSON
caminhoJSON = "Dados/dados_sensores.json"
json_dados = []

with open(caminhoJSON, "r", encoding = "utf-8") as arquivo:
    for linha in arquivo:
        json_dados.append(json.loads(linha))

# Inserindo os dados no MongoDB
resulto = db[colecao].insert_many(json_dados)
print("Documentos inseridos com sucesso.")

Banco de dados 'FabricaDB' criado com sucesso.
Coleção 'dados_sensores' criada com sucesso.
Documentos inseridos com sucesso.


## 3. Realizar as transformações necessárias

Nesta etapa, vou realizar a conversão do arquivo JSON para um DataFrame do Pandas e efetuar a transformação da coluna de timestamp.

In [3]:
# Criando o DataFrame

# Local de armazenamento do arquivo
bancoDB = conexao["FabricaDB"]  
colecaoDB = bancoDB["dados_sensores"]

# Obtendo os documentos da coleção
documentos = colecaoDB.find()

# Convertendo os documentos para uma lista
dados = list(documentos)

# Imprimindo o resultado
df = pd.DataFrame(dados)
df.head()

,_id,sensor_id,time,leitura
0,6568b78af694d88154454b90,1,1512099000000,14.9287
1,6568b78af694d88154454b91,2,1512099000000,10.8688
2,6568b78af694d88154454b92,3,1512099000000,14.3780
3,6568b78af694d88154454b93,4,1512099000000,11.2780
4,6568b78af694d88154454b94,1,1512099060000,12.5301


In [4]:
# Conversão da coluna "time"
df['time'] = pd.to_datetime(df['time'], unit = 'ms')

# Separando as informações em novas colunas
df['data'] = df['time'].dt.date
df['hora'] = df['time'].dt.time

# Reordenando as colunas
df_final = df[['_id', 'sensor_id', 'data', 'hora', 'leitura']]

# Imprimindo o resultado
df_final.head()

,_id,sensor_id,data,hora,leitura
0,6568b78af694d88154454b90,1,2017-12-01,03:30:00,14.9287
1,6568b78af694d88154454b91,2,2017-12-01,03:30:00,10.8688
2,6568b78af694d88154454b92,3,2017-12-01,03:30:00,14.3780
3,6568b78af694d88154454b93,4,2017-12-01,03:30:00,11.2780
4,6568b78af694d88154454b94,1,2017-12-01,03:31:00,12.5301


## 4. Disponibilizar o arquivo estruturado em uma tabela no SQL Server

In [5]:
# Conectando ao SQL Server

# Definindo o servidor e o banco de dados
server = 'DESKTOP-JTQE1FU'
banco = 'Fabrica'

# String de conexão
connection_string = f'Driver={{ODBC Driver 17 for SQL Server}};Server={server};DATABASE={banco};Trusted_Connection=yes'

# Realizando a conexão
conn = pyodbc.connect(connection_string)

In [6]:
# Inserção dos dados no SQL Server

# Criando um cursor
cursor = conn.cursor()

# Definindo o nome da tabela
nome_tabela = 'tb_fato_sensores'

# Query para verificação da existência da tabela
verifica_tabela = f"""
IF OBJECT_ID('{nome_tabela}', 'U') IS NOT NULL
    DROP TABLE {nome_tabela};
"""

# Executando a query
cursor.execute(verifica_tabela)

# Query para criação da tabela
cria_tabela = f"""
CREATE TABLE {nome_tabela} (
    _id VARCHAR(24),
    sensor_id INT,
    data DATE,
    hora VARCHAR(8),
    leitura FLOAT
);
"""

# Executando a query
cursor.execute(cria_tabela)

# Inserção dos dados
for indice, linha in df_final.iterrows():
    cursor.execute(f"""
    INSERT INTO {nome_tabela} (_id, sensor_id, data, hora, leitura)
    VALUES (?, ?, ?, ?, ?)
    """, str(linha['_id']), linha['sensor_id'], pd.to_datetime(linha['data']).date(), linha['hora'], linha['leitura'])

# Aplicando as alterações
conn.commit()

# Fechando a conexão
conn.close()
print(f"Dados inseridos na '{nome_tabela}' com sucesso.")

Dados inseridos na 'tb_fato_sensores' com sucesso.
